# **AlexNet Module** 

In [ ]:
import numpy as np
import tensorflow as tf

In [ ]:
# np.load(path, allow_pickle=True)
net_data = np.load("bvlc-alexnet.npy",allow_pickle=True, encoding="latin1").item()

In [ ]:
def conv(input, kernel, biases, k_h, k_w, c_o, s_h, s_w,  padding="VALID", group=1):
    '''
    From https://github.com/ethereon/caffe-tensorflow
    '''
    c_i = input.get_shape()[-1]
    assert c_i % group == 0
    assert c_o % group == 0
    convolve = lambda i, k: tf.nn.conv2d(i, k, [1, s_h, s_w, 1], padding=padding)
    if tf.__version__ < "1.0.0":
        if group == 1:
            conv = convolve(input, kernel)
        else:
            input_groups = tf.split(3, group, input)
            kernel_groups = tf.split(3, group, kernel)
            output_groups = [convolve(i, k) for i, k in zip(input_groups, kernel_groups)]
            conv = tf.concat(3, output_groups)
    else:
        if group == 1:
            conv = convolve(input, kernel)
        else:
            input_groups = tf.split(input, group, 3)
            kernel_groups = tf.split(kernel, group, 3)
            output_groups = [convolve(i, k) for i, k in zip(input_groups, kernel_groups)]
            conv = tf.concat(output_groups, 3)
    return tf.reshape(tf.nn.bias_add(conv, biases), [-1] + conv.get_shape().as_list()[1:])

In [ ]:
def AlexNet(features, feature_extract=False):
    """
    Builds an AlexNet model, loads pretrained weights
    """
    # conv1
    # conv(11, 11, 96, 4, 4, padding='VALID', name='conv1')
    k_h = 11
    k_w = 11
    c_o = 96
    s_h = 4
    s_w = 4
    conv1W = tf.Variable(net_data["conv1"][0])
    conv1b = tf.Variable(net_data["conv1"][1])
    conv1_in = conv(features, conv1W, conv1b, k_h, k_w, c_o, s_h, s_w, padding="SAME", group=1)
    conv1 = tf.nn.relu(conv1_in)

    # lrn1
    # lrn(2, 2e-05, 0.75, name='norm1')
    radius = 2
    alpha = 2e-05
    beta = 0.75
    bias = 1.0
    lrn1 = tf.nn.local_response_normalization(conv1, depth_radius=radius, alpha=alpha, beta=beta, bias=bias)

    # maxpool1
    # max_pool(3, 3, 2, 2, padding='VALID', name='pool1')
    k_h = 3
    k_w = 3
    s_h = 2
    s_w = 2
    padding = 'VALID'
    maxpool1 = tf.nn.max_pool(lrn1, ksize=[1, k_h, k_w, 1], strides=[1, s_h, s_w, 1], padding=padding)

    # conv2
    # conv(5, 5, 256, 1, 1, group=2, name='conv2')
    k_h = 5
    k_w = 5
    c_o = 256
    s_h = 1
    s_w = 1
    group = 2
    conv2W = tf.Variable(net_data["conv2"][0])
    conv2b = tf.Variable(net_data["conv2"][1])
    conv2_in = conv(maxpool1, conv2W, conv2b, k_h, k_w, c_o, s_h, s_w, padding="SAME", group=group)
    conv2 = tf.nn.relu(conv2_in)

    # lrn2
    # lrn(2, 2e-05, 0.75, name='norm2')
    radius = 2
    alpha = 2e-05
    beta = 0.75
    bias = 1.0
    lrn2 = tf.nn.local_response_normalization(conv2, depth_radius=radius, alpha=alpha, beta=beta, bias=bias)

    # maxpool2
    # max_pool(3, 3, 2, 2, padding='VALID', name='pool2')
    k_h = 3
    k_w = 3
    s_h = 2
    s_w = 2
    padding = 'VALID'
    maxpool2 = tf.nn.max_pool(lrn2, ksize=[1, k_h, k_w, 1], strides=[1, s_h, s_w, 1], padding=padding)

    # conv3
    # conv(3, 3, 384, 1, 1, name='conv3')
    k_h = 3
    k_w = 3
    c_o = 384
    s_h = 1
    s_w = 1
    group = 1
    conv3W = tf.Variable(net_data["conv3"][0])
    conv3b = tf.Variable(net_data["conv3"][1])
    conv3_in = conv(maxpool2, conv3W, conv3b, k_h, k_w, c_o, s_h, s_w, padding="SAME", group=group)
    conv3 = tf.nn.relu(conv3_in)

    # conv4
    # conv(3, 3, 384, 1, 1, group=2, name='conv4')
    k_h = 3
    k_w = 3
    c_o = 384
    s_h = 1
    s_w = 1
    group = 2
    conv4W = tf.Variable(net_data["conv4"][0])
    conv4b = tf.Variable(net_data["conv4"][1])
    conv4_in = conv(conv3, conv4W, conv4b, k_h, k_w, c_o, s_h, s_w, padding="SAME", group=group)
    conv4 = tf.nn.relu(conv4_in)

    # conv5
    # conv(3, 3, 256, 1, 1, group=2, name='conv5')
    k_h = 3
    k_w = 3
    c_o = 256
    s_h = 1
    s_w = 1
    group = 2
    conv5W = tf.Variable(net_data["conv5"][0])
    conv5b = tf.Variable(net_data["conv5"][1])
    conv5_in = conv(conv4, conv5W, conv5b, k_h, k_w, c_o, s_h, s_w, padding="SAME", group=group)
    conv5 = tf.nn.relu(conv5_in)

    # maxpool5
    # max_pool(3, 3, 2, 2, padding='VALID', name='pool5')
    k_h = 3
    k_w = 3
    s_h = 2
    s_w = 2
    padding = 'VALID'
    maxpool5 = tf.nn.max_pool(conv5, ksize=[1, k_h, k_w, 1], strides=[1, s_h, s_w, 1], padding=padding)

    # fc6, 4096
    fc6W = tf.Variable(net_data["fc6"][0])
    fc6b = tf.Variable(net_data["fc6"][1])
    flat5 = tf.reshape(maxpool5, [-1, int(np.prod(maxpool5.get_shape()[1:]))])
    fc6 = tf.nn.relu(tf.matmul(flat5, fc6W) + fc6b)

    # fc7, 4096
    fc7W = tf.Variable(net_data["fc7"][0])
    fc7b = tf.Variable(net_data["fc7"][1])
    fc7 = tf.nn.relu(tf.matmul(fc6, fc7W) + fc7b)

    if feature_extract:
        return fc7

    '''
    we use this layer when  AlexNet  trained on the ImageNet database, 
    which has 1000 classes of images
    '''
    # fc8, 1000 
    fc8W = tf.Variable(net_data["fc8"][0])
    fc8b = tf.Variable(net_data["fc8"][1])

    logits = tf.matmul(fc7, fc8W) + fc8b
    probabilities = tf.nn.softmax(logits)

    return probabilities

# **Imagenet_Inference**

In [ ]:
import time
import tensorflow as tf
import numpy as np
from matplotlib.pyplot import imread
# from caffe_classes import class_names

In [ ]:
"""
# x is a placeholders for a batch of input images.
x = tf.compat.v1.placeholder(tf.float32, (None, 227, 227, 3))

"""

'\n# x is a placeholders for a batch of input images.\nx = tf.compat.v1.placeholder(tf.float32, (None, 227, 227, 3))\n\n'

In [ ]:
"""

probs = AlexNet(x, feature_extract=False)
init = tf.compat.v1.global_variables_initializer()
sess = tf.compat.v1.Session()
sess.run(init)

# Read Images
im1 = (imread("poodle.png")[:, :, :3] * 255).astype(np.float32)
im1 = im1 - np.mean(im1)

im2 = (imread("weasel.png")[:, :, :3] * 255).astype(np.float32)
im2 = im2 - np.mean(im2)

# Run Inference
t = time.time()
output = sess.run(probs, feed_dict={x: [im1, im2]})

# Print Output
for input_im_ind in range(output.shape[0]):
    inds = np.argsort(output)[input_im_ind, :]
    print("Image", input_im_ind)
    for i in range(5):
        print("%s: %.3f" % (class_names[inds[-1 - i]], output[input_im_ind, inds[-1 - i]]))
    print()

print("Time: %.3f seconds" % (time.time() - t))

"""

'\n\nprobs = AlexNet(x, feature_extract=False)\ninit = tf.compat.v1.global_variables_initializer()\nsess = tf.compat.v1.Session()\nsess.run(init)\n\n# Read Images\nim1 = (imread("poodle.png")[:, :, :3] * 255).astype(np.float32)\nim1 = im1 - np.mean(im1)\n\nim2 = (imread("weasel.png")[:, :, :3] * 255).astype(np.float32)\nim2 = im2 - np.mean(im2)\n\n# Run Inference\nt = time.time()\noutput = sess.run(probs, feed_dict={x: [im1, im2]})\n\n# Print Output\nfor input_im_ind in range(output.shape[0]):\n    inds = np.argsort(output)[input_im_ind, :]\n    print("Image", input_im_ind)\n    for i in range(5):\n        print("%s: %.3f" % (class_names[inds[-1 - i]], output[input_im_ind, inds[-1 - i]]))\n    print()\n\nprint("Time: %.3f seconds" % (time.time() - t))\n\n'

# **Traffic_Sign_Inference**

In [ ]:
import time
import tensorflow as tf
import numpy as np
from imageio import imread
# from caffe_classes import class_names

In [ ]:
"""

# x is a placeholders for a batch of input images.
x = tf.compat.v1.placeholder(tf.float32, (None, 32, 32, 3))
#  resize the chape of the image to (227,227)
resized = tf.image.resize(x,(227,227))

"""

'\n\n# x is a placeholders for a batch of input images.\nx = tf.compat.v1.placeholder(tf.float32, (None, 32, 32, 3))\n#  resize the chape of the image to (227,227)\nresized = tf.image.resize(x,(227,227))\n\n'

In [ ]:
"""

probs = AlexNet(resized,feature_extract=False)

init = tf.compat.v1.global_variables_initializer()
sess = tf.compat.v1.Session()
sess.run(init)

# Read Images
im1 = imread("construction.jpg").astype(np.float32)
im1 = im1 - np.mean(im1)

im2 = imread("stop.jpg").astype(np.float32)
im2 = im2 - np.mean(im2)

# Run Inference
t = time.time()
output = sess.run(probs, feed_dict={x: [im1, im2]})

# Print Output
for input_im_ind in range(output.shape[0]):
    inds = np.argsort(output)[input_im_ind, :]
    print("Image", input_im_ind)
    for i in range(5):
        print("%s: %.3f" % (class_names[inds[-1 - i]], output[input_im_ind, inds[-1 - i]]))
    print()

print("Time: %.3f seconds" % (time.time() - t))

"""

'\n\nprobs = AlexNet(resized,feature_extract=False)\n\ninit = tf.compat.v1.global_variables_initializer()\nsess = tf.compat.v1.Session()\nsess.run(init)\n\n# Read Images\nim1 = imread("construction.jpg").astype(np.float32)\nim1 = im1 - np.mean(im1)\n\nim2 = imread("stop.jpg").astype(np.float32)\nim2 = im2 - np.mean(im2)\n\n# Run Inference\nt = time.time()\noutput = sess.run(probs, feed_dict={x: [im1, im2]})\n\n# Print Output\nfor input_im_ind in range(output.shape[0]):\n    inds = np.argsort(output)[input_im_ind, :]\n    print("Image", input_im_ind)\n    for i in range(5):\n        print("%s: %.3f" % (class_names[inds[-1 - i]], output[input_im_ind, inds[-1 - i]]))\n    print()\n\nprint("Time: %.3f seconds" % (time.time() - t))\n\n'

# **Feature_Extraction**

In [ ]:
!pip install imageio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import time
import numpy as np
import pandas as pd
from imageio import imread
import tensorflow as tf
tf.compat.v1.disable_eager_execution()


In [ ]:
"""

#Load the signname csv file
sign_names = pd.read_csv('signnames.csv')
nb_classes = 43

"""

"\n\n#Load the signname csv file\nsign_names = pd.read_csv('signnames.csv')\nnb_classes = 43\n\n"

x is a placeholder for a batch of input images.

In [ ]:
"""

x = tf.compat.v1.placeholder(tf.float32, (None, 32, 32, 3))
resized = tf.compat.v1.image.resize_images(x, (227, 227))

"""

'\n\nx = tf.compat.v1.placeholder(tf.float32, (None, 32, 32, 3))\nresized = tf.compat.v1.image.resize_images(x, (227, 227))\n\n'

In [ ]:
"""

# NOTE: By setting `feature_extract` to `True` we return
# the second to last layer.
fc7 = AlexNet(resized, feature_extract=True)
# Last fully connected layer
shape = (fc7.get_shape().as_list()[-1], nb_classes)  
fc8W = tf.Variable(tf.compat.v1.truncated_normal(shape=shape,  stddev = 1e-1))
fc8b = tf.Variable(tf.zeros(nb_classes))
logits = tf.matmul(fc7, fc8W) + fc8b
# softmax activation function
probs = tf.nn.softmax(logits)

init = tf.compat.v1.global_variables_initializer()
sess = tf.compat.v1.Session()
sess.run(init)

"""

'\n\n# NOTE: By setting `feature_extract` to `True` we return\n# the second to last layer.\nfc7 = AlexNet(resized, feature_extract=True)\n# Last fully connected layer\nshape = (fc7.get_shape().as_list()[-1], nb_classes)  \nfc8W = tf.Variable(tf.compat.v1.truncated_normal(shape=shape,  stddev = 1e-1))\nfc8b = tf.Variable(tf.zeros(nb_classes))\nlogits = tf.matmul(fc7, fc8W) + fc8b\n# softmax activation function\nprobs = tf.nn.softmax(logits)\n\ninit = tf.compat.v1.global_variables_initializer()\nsess = tf.compat.v1.Session()\nsess.run(init)\n\n'

In [ ]:
"""

# Read Images
im1 = imread("construction.jpg").astype(np.float32)
im1 = im1 - np.mean(im1)

im2 = imread("stop.jpg").astype(np.float32)
im2 = im2 - np.mean(im2)

# Run Inference
t = time.time()
output = sess.run(probs, feed_dict={x: [im1, im2]})

# Print Output
for input_im_ind in range(output.shape[0]):
    inds = np.argsort(output)[input_im_ind, :]
    print("Image", input_im_ind)
    for i in range(5):
        print("%s: %.3f" % (sign_names.loc[inds[-1 - i]][1], output[input_im_ind, inds[-1 - i]]))
    print()

print("Time: %.3f seconds" % (time.time() - t))

"""

'\n\n# Read Images\nim1 = imread("construction.jpg").astype(np.float32)\nim1 = im1 - np.mean(im1)\n\nim2 = imread("stop.jpg").astype(np.float32)\nim2 = im2 - np.mean(im2)\n\n# Run Inference\nt = time.time()\noutput = sess.run(probs, feed_dict={x: [im1, im2]})\n\n# Print Output\nfor input_im_ind in range(output.shape[0]):\n    inds = np.argsort(output)[input_im_ind, :]\n    print("Image", input_im_ind)\n    for i in range(5):\n        print("%s: %.3f" % (sign_names.loc[inds[-1 - i]][1], output[input_im_ind, inds[-1 - i]]))\n    print()\n\nprint("Time: %.3f seconds" % (time.time() - t))\n\n'

# **Train_Feature_Extraction**

In [ ]:
import pickle
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

In [ ]:
# Load traffic signs data.
training_File = "train.p"

with open(training_File, mode='rb') as f:
    train = pickle.load(f)

# Split data into training and validation sets.
X_train, X_valid,y_train,y_valid = train_test_split(train['features'], train['labels'],test_size=0.2, random_state=0)

# Define placeholders and resize operation.
x = tf.compat.v1.placeholder(tf.float32, (None, 32, 32, 3))
labels = tf.compat.v1.placeholder(tf.int64, None)
resized = tf.compat.v1.image.resize_images(x, (227, 227))

fc7 = AlexNet(resized, feature_extract=True)
# `tf.stop_gradient` prevents the gradient from flowing backwards
# past this point, keeping the weights before and up to `fc7` frozen.
# This also makes training faster, less work to do!
fc7 = tf.stop_gradient(fc7)

# the final layer for traffic sign classification.
nb_classes = 43
shape = (fc7.get_shape().as_list()[-1], nb_classes)  
fc8W = tf.Variable(tf.compat.v1.truncated_normal(shape=shape,  stddev = 1e-2))
fc8b = tf.Variable(tf.zeros(nb_classes))
logits = tf.matmul(fc7, fc8W) + fc8b


# Define loss, training, accuracy operations.

cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits( logits=logits, labels=labels)
loss_operation = tf.reduce_mean(cross_entropy)
optimizer = tf.compat.v1.train.AdamOptimizer()
training_operation = optimizer.minimize(loss_operation,var_list=[fc8W,fc8b])
init_op = tf.compat.v1.initialize_all_variables()


predict_operation = tf.argmax(logits, 1)
correct_prediction = tf.equal(predict_operation, labels)
accuracy_operation = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

# Train and evaluate the feature extraction model.
#Evaluate function
def evaluate(X, y):
    total_acc = 0
    num_examples = len(X)
    total_loss = 0
    sess = tf.compat.v1.get_default_session()
    for offset in range(0, num_examples, batch_size):
        end = offset + batch_size
        X_batch = X[offset:end]
        y_batch = y[offset:end]

        loss, acc = sess.run([loss_operation, accuracy_operation], feed_dict={x: X_batch, labels: y_batch})
        total_loss += (loss * len(X_batch))
        total_acc += (acc * len(X_batch))

    return total_loss/num_examples, total_acc/num_examples

#Hyper parameters
epochs=10
batch_size=128
#Train the alexnet 
with tf.compat.v1.Session() as sess:
    sess.run(init_op)
    print("Start Training...")
    print("Number of epochs : {}".format(epochs))
    print("Batch size : {}".format(batch_size))

    for i in range(epochs):
        # training
        X_train, y_train = shuffle(X_train, y_train)
        for offset in range(0, X_train.shape[0], batch_size):
            end = offset + batch_size
            batch_x,batch_y=X_train[offset:end], y_train[offset:end]
            sess.run(training_operation, feed_dict={x:batch_x , labels:batch_y})

        val_loss, val_acc = evaluate(X_valid, y_valid)
        print("Epoch", i+1)
        print("Validation Loss =", val_loss)
        print("Validation Accuracy =", val_acc)
        print("")

Start Training...
Number of epochs : 10
Batch size : 128
Epoch 1
Validation Loss = 0.4789389237035631
Validation Accuracy = 0.8718439173224142

Epoch 2
Validation Loss = 0.3270055975302298
Validation Accuracy = 0.915200204059987

Epoch 3
Validation Loss = 0.2349580618244323
Validation Accuracy = 0.9422341238415627

Epoch 4
Validation Loss = 0.19906932551074594
Validation Accuracy = 0.947717418919349

Epoch 5
Validation Loss = 0.17965733690305621
Validation Accuracy = 0.9528181585113906

Epoch 6
Validation Loss = 0.15288589760999355
Validation Accuracy = 0.9599591940983434

Epoch 7
Validation Loss = 0.13163596833278196
Validation Accuracy = 0.9654424890393172

Epoch 8
Validation Loss = 0.13133854480368604
Validation Accuracy = 0.9671002295332823

Epoch 9
Validation Loss = 0.12007146757936879
Validation Accuracy = 0.969140525391381

Epoch 10
Validation Loss = 0.11379378693531253
Validation Accuracy = 0.9693955623717431

